In [1]:
!pip install pulp

In [1]:
from pulp import *
import pandas as pd
import ipywidgets as widgets
import numpy as np
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

### Importing Data

In [3]:
lp_data = pd.read_pickle("./INPUT/lp_data.pkl")
final_packs = pd.read_pickle("./INPUT/final_packs.pkl")

In [4]:
#Dictionary of pack prices to use for the constraint 
smry = final_packs.groupby('pack_name')['processed_price_float'].first().reset_index(drop=False) 
pack_prices_dict = {}
for idx, row in smry.iterrows():
    pack_prices_dict[row['pack_name']] = row['processed_price_float']

In [5]:
ch_selector = widgets.SelectMultiple(
    options=lp_data['Channel Name'],
    value=['Zoom'],
    description='Selectable Channels',
    disabled=False
)

display(ch_selector)

SelectMultiple(description='Selectable Channels', index=(0,), options=('Zoom', 'News18 Urdu', 'Zee News', 'Zee…

In [8]:
lp_channels = list(ch_selector.value)
lp_data[lp_data['Channel Name'].isin(lp_channels)].shape

(129, 4)

In [9]:
prob = pulp.LpProblem('Minimizing-Tata-Sky-spend', pulp.LpMinimize)

### Lists of decision variables and constraints  
decision_variables = [] 
constraints = []

ch_map = {} # Dictionary to map between variable names - x0, x1 - and the channels they represent
pack_map = {} # Dictionary to map between pack variable names - c0, c1 - and the packs they represent
rev_pack_map = {} # Reverse of the above dictionary 

ch_count = 0
pack_count = 0 


total_cost = ""

### Iterate over all selected channels and add them, and the packs that contain 
### said channel, to the objective and constraint functions 
for idx,x in lp_data.loc[lp_data['Channel Name'].isin(lp_channels)].iterrows():
    temp_contraint = ""
    
    #Creating variables for standalone channels 
    variable_str = str('x' + str(ch_count))
    ch_count += 1
    variable = pulp.LpVariable(str(variable_str), lowBound = 0, upBound = 1, cat= 'Integer')
    decision_variables.append(variable)
    ch_map.update({variable_str : x['Channel Name']})
    
    #Adding standalone channel to cost 
    formula = x['MRP_Tax']*variable
    total_cost += formula
    
    # Adding channel
    temp_contraint += 1*variable
    
    
    
    for p in x['Packs']:
        
        ### Check if pack already exists. This is important because if both x0 and x1 are contained in
        ### a particular pack, we don't want to create two seperate pack decision variables for it. 
        if p[0] not in rev_pack_map:
            
            # Creating variable for packs 
            variable_str = str('c' + str(pack_count))
            pack_count += 1
            variable = pulp.LpVariable(str(variable_str), lowBound = 0, upBound = 1, cat= 'Integer')
            decision_variables.append(variable)
            pack_map.update({variable_str : p[0]})
            rev_pack_map.update({p[0]:variable_str})
            
            # Adding pack cost to cost 
            formula = pack_prices_dict[p[0]]*variable # Look up the pack price dictionary for... the price, naturally. 
            total_cost += formula
            
            # Adding pack as constraint 
            temp_contraint += 1*variable
        
        ### If pack decision variable has already been created, we use it instead 
        else:
            t_var = [x for x in decision_variables if x.__str__() == rev_pack_map[p[0]]][0] # get lp variable object
            temp_contraint += 1*t_var
            
    constraints.append(temp_contraint) # Adding constraint for a particular channel to the entire set of constraints


In [10]:
prob += total_cost # Add objective function to the model 
for c in constraints: # Add constraints individually to the model 
    prob += (c >= 1) 

In [11]:
optimization_result = prob.solve() 

In [45]:
all_channels_df = pd.DataFrame() # Some basic validation. If a standalone channel is already in the pack, that implies 
# the result is not optimal. 
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", round(value(prob.objective),2))
print ("Individual decision_variables: ")
for v in prob.variables():
    if v.varValue == 1:
        if v.name in pack_map.keys():
            print("PACK -- ",pack_map[v.name], "=", v.varValue)
            all_channels_df = all_channels_df.append(final_packs[final_packs['pack_name']== pack_map[v.name]])
            
for v in prob.variables():
    if v.varValue == 1:
        if v.name not in pack_map.keys():
            print(ch_map[v.name], "=", v.varValue)
            if all_channels_df[all_channels_df['Channel/Service Name'].isin([ch_map[v.name]])].shape[0] > 0:
                print(f"Error!!! : {ch_map[v.name]}")

Status: Optimal
Optimal Solution to the problem:  737.05
Individual decision_variables: 
PACK --  Hindi News = 1.0
PACK --  Hindi Movies Mini = 1.0
PACK --  Kids = 1.0
PACK --  Knowledge & Lifestyle = 1.0
PACK --  Hindi Entertainment Mini = 1.0
PACK --  English News = 1.0
Zoom = 1.0
News18 Urdu = 1.0
Baby TV HD = 1.0
&TV = 1.0
WB = 1.0
News18 Assam North East = 1.0
Star Plus HD = 1.0
Zee TV HD = 1.0
Aaj Tak HD = 1.0
Star Sports 1 HD = 1.0
Star Gold HD = 1.0
SET = 1.0
Colors = 1.0
News18 Kerala = 1.0
Sony Aath = 1.0
Star Sports 1 = 1.0
Colors Cineplex HD = 1.0
Sony Max HD = 1.0
Colors HD = 1.0
SET HD = 1.0
Zee Cinema HD = 1.0
&pictures HD = 1.0
Sony SAB HD = 1.0
DSport = 1.0
Zee ETC = 1.0
Zing = 1.0
Sony Mix = 1.0
History TV18 HD = 1.0
Nick HD+ = 1.0
Zee Salaam = 1.0
Nat Geo Wild HD = 1.0
National Geographic HD = 1.0
News18 Bihar Jharkhand = 1.0
Living Foodz HD = 1.0
Star Sports 2 = 1.0
Star Sports First = 1.0
Star Sports 2 HD = 1.0
Vh1 = 1.0
VH1 HD = 1.0
Animal Planet HD World = 1.0
TL